In [2]:
//@ts-check
var ohm = require('ohm-js');
var nj = require('numjs');

/**
 * @param {string} s
 * */
function parse_brackets(s) {
  let i = 0;
  function parse(expect = '', skip = 0) {
    let start = i = i + skip;
    while (i < s.length && s[i] != '(' && s[i] != '[') {
      if (s[i] == expect) break;
      else if (s[i] == ')' || s[i] == ']') throw new Error(`unexpected "${s[i]}" (position ${i} of ${s})`);
      i++;
    }
    if (i == s.length && expect == '') return [s.slice(start)];
    else if (i == s.length) throw new Error(`expected "${expect}" not found`);
    if (s[i] == '(') return [s.slice(start, i), '()', [parse(')', 1)], ...parse(expect)];
    if (s[i] == '[') return [s.slice(start, i), '[]', [parse(']', 1)], ...parse(expect)];
    i += 1;
    return [s.slice(start, i - 1)];
  }
  const out = [];
  while (i < s.length) out.push(...parse());
  return out;
}

parse_brackets(`a[idx[mask==(w^~z)]] = (sum(b+1, axis=0) * c).mean()`)

[
  'a',
  '[]',
  [ [ 'idx', '[]', [Array], '' ] ],
  ' = ',
  '()',
  [ [ 'sum', '()', [Array], ' * c' ] ],
  '.mean',
  '()',
  [ [ '' ] ],
  ''
]

In [3]:

class MyArray {
  constructor(flat, shape, size = null) {
    this.flat = flat;
    this.shape = shape;
    this.size = size != null ? size : reduce(shape, (a, b) => a * b, 1);
  }
  to_list(){
    function recursiveReshape(flatArr, shapeArr) {
      if (shapeArr.length === 0) {
        return flatArr.shift();
      }
      const innerShape = shapeArr.slice(1);
      const outerSize = shapeArr[0];
      const innerArray = [];
      for (let i = 0; i < outerSize; i++) {
        innerArray.push(recursiveReshape(flatArr, innerShape));
      }
      return innerArray;
    }
    return recursiveReshape(this.flat, this.shape);
  }
}

In [10]:

myGrammar = ohm.grammar(String.raw`
NumpyGrammar {
  Main
  = Arr AssignSymbol Arr -- assignment
  | Arr                  -- expression
  
  Arr
    = Arr "[" Slice "]"         -- slice
    | Arr "." Name CallArgs     -- method
    | Name ("." Name)* CallArgs -- call
    | Arr "." Name              -- attribute
    | ArithmeticExp
   
  PriExp
    = "(" Arr ")"  -- parent
    |  "-" PriExp   -- neg
    |  "+" PriExp   -- pos
    |  Variable
    |  number
  
  Variable
   = "#" digit+ "#"
  
  AssignSymbol
  ="="|"+="|"-="|"/="|"%="|"&="|"|="|"^="|"@="
    
  ArithmeticExp
   = AddExp
    
  AddExp
    = AddExp "+" MulExp  -- plus
    | AddExp "-" MulExp  -- minus
    | MulExp

  MulExp
    = MulExp "*" ExpExp  -- times
    | MulExp "/" ExpExp  -- divide
    | ExpExp

  ExpExp
    = ExpExp "**" ExpExp  -- power
    | PriExp
   
  Name  (an identifier)
    = (letter|"_") (letter|"_"|digit)*

  number  (a number)
    = digit* "." digit+  -- fract
    | digit+             -- whole

  CallArgs // Using empty strings instead of separate rules
   = "(" Args ","  KwArgs ","? ")"
   | "(" Args ","? ""      ""  ")"
   | "(" ""   ","? KwArgs ","? ")"
   | "(" ""    ""  ""      ""  ")"
   
  Args = NonemptyListOf<ArgValue, ",">
  KwArgs = NonemptyListOf<KwArg, ",">
  KwArg = Name "=" ArgValue
  ArgValue = String | "True" | "False" | "None" | Arr
  String = "\'" any* "\'" | "\"" any* "\""
   
  Slice = NonemptyListOf<SliceTerm, ",">
  SliceTerm = ":" | "None" | Arr 
}
`);

// m = myGrammar.match('sum(#.mean(axis=0), #*#, axis=-1)');
// console.log(m.succeeded());


log = (x) => (console.log("ARR", x), x);
semantics = myGrammar.createSemantics();
semantics.addOperation('eval', {
  number(digits) {
    return parseInt(digits.sourceString)
  },
  Arr_slice($arr, _open, $slice, _close) {
    return $arr.eval()[$slice.eval()];
  },
  Arr_call($name, $names, _, $callArgs) {
    let name = $name.sourceString + $names.sourceString;
    if (name.slice(0, 3) == "np.") name = name.slice(3);
    const func = MyArray.prototype[name];
    if (func === undefined) throw new Error(`Unrecognized function ${name}`)
    const { args, kwArgs } = $callArgs.eval();
    return func(kwArgs, ...args);
  },
  Variable(_, $i, __) {
    const i = parseInt($i.sourceString);
    const arr = semantics.variables[i];
    return arr;
  },
  Main_assignment($tgt, $slice, $src) {
    const tgt = $tgt.eval();
    const slice = $slice.eval();
    const src = $src.eval();
    throw 'not implemented';
    return null;
  },
  Main_expression($x) {
    const x = $x.eval();
    return x.to_list();
  },
  CallArgs(_open, $args, _comma, $kwArgs, _trailing, _close) {
    const args = $args.eval();
    let kwArgs = Object.fromEntries($kwArgs.eval() || []);
    return { args, kwArgs };
  },
  KwArg($key, _equals, $value) {
    const key = $key.sourceString;
    const value = $value.sourceString;
    return [key, value];
  },
  ArgValue($obj) {
    switch ($obj) {
      case "True": return true;
      case "False": return false;
      case "None": return null;
    }
    return $obj.eval();
  },

  NonemptyListOf(first, _, more) {
    return [first, ...more.children].map(c => c.eval());
  },
  _terminal() { return null; },
});

/**
 * @param {TemplateStringsArray} htmlTemplateString
 * @param {any[]} variables
 * */
function np(htmlTemplateString, ...variables) {
  // Make a tree of operations
  // Join with #0# #1# #2#, ...
  // First by parenthesis () and square brackets []
  let idx = 0;
  const str = htmlTemplateString.join('###').replace(/###/g, () => `#${idx++}#`);
  // console.log(str);
  semantics.variables = variables;
  const match = myGrammar.match(str);
  if (!match.succeeded()) throw new Error(`Syntax error in string: ${str}`);
  return semantics(match).eval();
}

function generateCombinations(inputArray) {
  const combinations = [];
  const max = inputArray.length;
  function recursion(currentCombination, index) {
    if (index === max) return combinations.push([...currentCombination]);
    for (let i = 0; i < inputArray[index]; i++) {
      currentCombination[index] = i;
      recursion(currentCombination, index + 1);
    }
  }
  recursion(new Array(max).fill(0), 0);
  return combinations;
}

MyArray.prototype._new = function (shape, f) {
  if (!Array.isArray(shape)) shape = [shape];
  const size = shape.reduce((a, b) => a * b, 1);
  const flat = Array.from({ length: size }, f)
  return new MyArray(flat, shape, size);
};
MyArray.prototype.zeros = function (_, shape) {
  return MyArray.prototype._new(shape, (_) => 0)
};
MyArray.prototype.ones = function (_, shape) {
  return MyArray.prototype._new(shape, (_) => 1)
};
MyArray.prototype.arange = function (_, arg0, arg1) {
  let start, end;
  if (arg1 === undefined) start = 0, end = arg0;
  else start = arg0, end = arg1;
  return MyArray.prototype._new(end - start, (_, i) => start + i)
};
MyArray.prototype.sum = function ({ axis, keepdims } = { axis: null, keepdims: false }, arr) {
  return MyArray.prototype._reduce(axis, keepdims, (a, b) => a + b, 0, arr);
};
MyArray.prototype._reduce = function (axis, keepdims, func, defaultValue, arr) {
  if (axis == null) return arr.flat.reduce(func, defaultValue);
  if (axis < 0) axis = arr.shape.length - 1;
  let q = arr.size / arr.shape[axis];
  let flat = [];
  for(let i=0; i < q; i++) {
    const sub = [];
    for (let j = i; j < arr.size; j += q) sub.push(arr.flat[j]);
    flat.push(sub.reduce(func, defaultValue));
  }
  let shape = [...arr.shape];
  if (keepdims) shape[axis] = 1;
  else shape = shape.filter((_, i) => i != axis);
  return new MyArray(flat, shape, arr.size / q);
};

// semantics('sum(#.mean(axis=0), #*#, axis=-1)').eval()
// np`${[]} = np.sum(${[]}) * ${0}`
// np`sum(${[]}.mean(axis=0), (${[]}*${[]}).var(keepdims=True, axis=None), axis=-1)`

// np`np.zeros(3)`
np`np.zeros(${[3, 5, 4]})`
np`np.sum(np.ones(${[10, 3, 2]}), axis=0, keepdims=1)`

// 
// a = [1, 2, 3, 4]
// np`${a}[:] = 3`
// a



[ [ [ 10, 10 ], [ 10, 10 ], [ 10, 10 ] ] ]

In [5]:


// Example usage:
flatArray = [1, 2, 3, 4, 5, 6];
shape = [3, 2, 1, 1]; // Represents a 2x3 matrix
multiDimArray = reshapeArray(flatArray, shape);

console.log(multiDimArray);

ReferenceError: reshapeArray is not defined